# Database Filtering

- We have a list of subject id's that represent patients with a pregnancy diagnosis that we want to use to filter the database so that we are working with a smaller set of patients
- The non-pregnant patients (and their data) are not relevant to this analysis, so they should be filtered out
- In order to be cautious, we will be creating new tables rather than directly modifying the existing tables (the existing tables took hours to upload to postgres and are useful to have as a backup)
- **Filter all “relevant” tables (based on analysis of relevance for our study) by the subject id list – will include all relevant rows for each of the pregnant subjects**

In [14]:
import psycopg2
from psycopg2 import OperationalError, DatabaseError, sql
import pandas
import csv

In [15]:
### Environment Variables for Connection ###
DB_NAME = 'smcdougall'
USERNAME = 'postgres'
PASSWORD = 'postgres'
HOST = 'localhost'
PORT = 5432 

In [16]:
def connect_to_postgres(db_name, username, password, host, port):
    connection = None
    try:
        connection = psycopg2.connect(
            dbname=db_name,
            user=username,
            password=password,
            host=host,
            port=port
        )
        print('Connected to db:', db_name)
        return connection
    except OperationalError as e:
        print('Received the following error:', e)
        return None

In [17]:
def verify_postgres_connection(connection):
    if connection is not None:
        try:
            cur = connection.cursor()
            cur.execute('SELECT version();')
            db_version = cur.fetchone()
            print('The Postgres database version is:', db_version)
            cur.close()
        except DatabaseError as e:
            print('Received the following error:', e)
    else:
        print('Connection to Postgres failed.')

In [18]:
def close_connection(connection):
    if connection is not None:
        connection.close()
        print('Postgres connection has been closed.')

In [19]:
connection = connect_to_postgres(DB_NAME, USERNAME, PASSWORD, HOST, PORT)
verify_postgres_connection(connection)
close_connection(connection)

Connected to db: smcdougall
The Postgres database version is: ('PostgreSQL 14.5 on aarch64-apple-darwin20.6.0, compiled by Apple clang version 12.0.5 (clang-1205.0.22.9), 64-bit',)
Postgres connection has been closed.


In [20]:
def load_csv_column_as_list(file_path):
    column_values = []
    with open(file_path, newline='') as csv_file:
        reader = csv.reader(csv_file)
        for row in reader:
            if row:  # Check if row is not empty
                column_values.append(row[0])
    return column_values

file_path = "hosp_and_ed_pregnant_subjects.csv"
# remove column header from the list since we are only interested in the actual ids
hosp_and_ed_subjects = load_csv_column_as_list(file_path)[1:]

The `hosp_and_ed_subjects` list contains the subject ids that we are interested in:

In [21]:
print(hosp_and_ed_subjects[:5])

['10000719', '10001319', '10001472', '10001884', '10002266']


All the other tables represent the id's as integers, so we will coerce the string id's to integers:

In [22]:
hosp_and_ed_subjects = [int(sub) for sub in hosp_and_ed_subjects]
print(hosp_and_ed_subjects[:5])

[10000719, 10001319, 10001472, 10001884, 10002266]


In [23]:
print(len(hosp_and_ed_subjects))

19088


In [24]:
# these pats come from the analysis completed in the data_preprocessing_notebook
deceased_pats_to_exclude = [10495653, 11611136, 10504589, 11101737, 15047583, 15014156, 15695321, 19017858, 18805396, 18892314, 17809756, 18186302]

In [25]:
hosp_and_ed_subjects = [sub for sub in hosp_and_ed_subjects if sub not in deceased_pats_to_exclude]
print(len(hosp_and_ed_subjects))

19076


## Relevant Tables for Filtering
From an earlier analysis, we have determined that there are some tables that are not relevant to this analysis at all, and so filtering will not be applied to them and they will not be used. On the other hand, of the relevant tables, most of them use `subject_id` as a linking field, **but not all of them**. Even though some of the tables don't use `subject_id` as the best linking field, all of them do have `subject_id` as a field, so they can all be filtered accordingly and then linked according to the ideal linking field.

In [26]:
# tables that contain subject_id field and that we want to filter
subject_id_filtered_tables = [
    # Hosp tables
    "mimiciv_hosp.omr",
    "mimiciv_hosp.admissions",
    "mimiciv_hosp.diagnoses_icd",
    "mimiciv_hosp.drgcodes",
    "mimiciv_hosp.emar",
    "mimiciv_hosp.emar_detail",
    "mimiciv_hosp.microbiologyevents",
    "mimiciv_hosp.patients",
    "mimiciv_hosp.pharmacy",
    "mimiciv_hosp.poe",
    "mimiciv_hosp.poe_detail",
    "mimiciv_hosp.prescriptions",
    "mimiciv_hosp.procedures_icd",
    "mimiciv_hosp.services",
    "mimiciv_hosp.transfers",
    "mimiciv_hosp.labevents",
    # ICU tables
    "mimiciv_icu.chartevents",
    "mimiciv_icu.datetimeevents",
    "mimiciv_icu.icustays",
    "mimiciv_icu.ingredientevents",
    "mimiciv_icu.inputevents",
    "mimiciv_icu.outputevents",
    "mimiciv_icu.procedureevents",
    # ED tables
    "mimiciv_ed.diagnosis",
    "mimiciv_ed.edstays",
    "mimiciv_ed.medrecon",
    "mimiciv_ed.pyxis",
    "mimiciv_ed.triage",
    "mimiciv_ed.vitalsign",
    # Notes tables
    "mimiciv_note.discharge",
    "mimiciv_note.discharge_detail",
    "mimiciv_note.radiology",
    "mimiciv_note.radiology_detail"
]

In [27]:
def filter_table_on_subject_id(connection, table_name, subject_id_list):
    cur = connection.cursor()

    try:
        # create new table based on the original table that is filtered based on subject id
        schema_name, base_table_name = table_name.split('.')
        filtered_table_name = f"{schema_name}.filtered_{base_table_name}"
        # use sql.Identifier to safely handle table names to prevent SQL injection
        query = sql.SQL("""
            DROP TABLE {};
            CREATE TABLE {}
            AS
            SELECT * FROM {}.{}
            WHERE subject_id IN %s
        """).format(
            sql.Identifier(filtered_table_name),
            sql.Identifier(filtered_table_name),
            sql.Identifier(schema_name),
            sql.Identifier(base_table_name)
        )
        cur.execute(query, (tuple(subject_id_list),))
        # commit the transaction after creating the new table
        connection.commit()
    
        print(f"Filtered table {filtered_table_name} created successfully!")

        # count number of rows in the filtered table
        query_rows = sql.SQL("""
            SELECT COUNT(*) FROM {}
        """).format(sql.Identifier(filtered_table_name))

        cur.execute(query_rows)
        num_rows = cur.fetchone()[0]
        print(f"Number of rows in filtered table {filtered_table_name}:", num_rows)

        # count unique subject Id's in the filtered table
        query_subject_ids = sql.SQL("""
            SELECT COUNT(DISTINCT subject_id) FROM {}
        """).format(sql.Identifier(filtered_table_name))

        cur.execute(query_subject_ids)
        num_unique_subject_ids = cur.fetchone()[0]
        print(f"Number of unique subject ids:", num_unique_subject_ids)
    
    except psycopg2.Error as e:
        print(f"Error creating table {filtered_table_name}:", e)
    finally:
        cur.close()

In [28]:
connection = connect_to_postgres(DB_NAME, USERNAME, PASSWORD, HOST, PORT)

for table_name in subject_id_filtered_tables:
    filter_table_on_subject_id(connection, table_name, hosp_and_ed_subjects)
close_connection(connection)

Connected to db: smcdougall
Filtered table mimiciv_hosp.filtered_omr created successfully!
Number of rows in filtered table mimiciv_hosp.filtered_omr: 978756
Number of unique subject ids: 15772
Filtered table mimiciv_hosp.filtered_admissions created successfully!
Number of rows in filtered table mimiciv_hosp.filtered_admissions: 56822
Number of unique subject ids: 17841
Filtered table mimiciv_hosp.filtered_diagnoses_icd created successfully!
Number of rows in filtered table mimiciv_hosp.filtered_diagnoses_icd: 608999
Number of unique subject ids: 17840
Filtered table mimiciv_hosp.filtered_drgcodes created successfully!
Number of rows in filtered table mimiciv_hosp.filtered_drgcodes: 91785
Number of unique subject ids: 17559
Filtered table mimiciv_hosp.filtered_emar created successfully!
Number of rows in filtered table mimiciv_hosp.filtered_emar: 2856872
Number of unique subject ids: 10842
Filtered table mimiciv_hosp.filtered_emar_detail created successfully!
Number of rows in filtered